In [1]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_name = "hotchpotch/youri-7b-sft-qa-context-jaqket-awq"

model = AutoAWQForCausalLM.from_quantized(
    model_name,
    fuse_layers=True,
    use_flash_attention_2=True,
    device_map="auto",
    force_download=True,
    resume_download=False,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

/home/yu1/miniconda3/envs/llm-sc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fusing layers...: 100%|██████████| 32/32 [00:00<00:00, 54.32it/s]


In [2]:
import torch


def get_cuda_current_memory_allocated():
    return int(torch.cuda.memory_allocated() / 1024 / 1024)


print("load model cuda memory(MB):", get_cuda_current_memory_allocated())

load model cuda memory(MB): 5933


In [3]:
RESPONSE_MESSAGE = "応答"
RESPONSE_PROMPT = f"### {RESPONSE_MESSAGE}: \n"
DEVICE = "cuda:0"


def build_prompt(
    user_message: str,
    inputs: str | None = "",
    separator: str = "\n\n### ",
    response_message: str = RESPONSE_MESSAGE,
) -> str:
    system_message = "以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。"
    prompt = system_message
    roles = ["指示", response_message]
    messages = [": \n" + user_message, ": \n"]

    if inputs:
        roles = ["指示", "入力", response_message]
        messages = [": \n" + user_message, ": \n" + inputs, ": \n"]

    for role, message in zip(roles, messages):
        prompt += separator + role + message
    return prompt


def qa(model, tokenizer, question, context, build_prompt_fn=build_prompt):
    prompt = build_prompt_fn(question, context)
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

    with torch.no_grad():
        output_ids = model.generate(
            input_ids=token_ids.to(DEVICE),  # type: ignore
            max_new_tokens=24,
            do_sample=False,
            # temperature=0.0,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    output = tokenizer.decode(output_ids.tolist()[0])
    # prompt を取り除く
    output = output.replace(prompt, "")
    # eos_token 以降を取り除く
    output = output.split(tokenizer.eos_token)[0]
    return output.strip()

In [4]:
question = "楽曲『約束はいらない』でデビューした、声優は誰？"
context = """
(0.21018645, '約束はいらない', '「約束はいらない」(やくそくはいらない)は、坂本真綾のデビューシングル。') (0.24241784, '約束はいらない', '坂本真綾の歌手デビュー作品。当時坂本はまだ無名の声優であったが、同曲がテーマソングとなったアニメ『天空のエスカフローネ』とともに知名度を上げることとなる。後に「指輪」が同アニメの劇場版映画の主題歌とな') (0.2441598, '坂本真綾', '本格的な歌手活動は、1996年にテレビアニメ『天空のエスカフローネ』へ出演したことがきっかけで始めており、同作のオープニングテーマソングである「約束はいらない」(岩里祐穂作詞、菅野よう子作曲)をシング') (0.26043588, 'Suara', '大阪府立豊中高等学校を経て大阪外国語大学でインドネシア語を専攻中にバンド・ユニットを組んで音楽活動を始めた。普段はお笑い番組が大好きなこともあってよく喋るほうだが、東京の仕事で標準語の喋りをする時は、') (0.2690969, '早見沙織', '声優デビュー時より、数多くの主題歌やキャラクターソングを担当し、バラードからポップス、ヒットソングのカバー曲や英語の楽曲など、様々な曲を歌いこなす。2009年には吉田仁美とのユニット「blue dro')
"""

print(qa(model, tokenizer, question, context))

坂本真綾


In [5]:
import pandas as pd
import datasets

ds = datasets.load_dataset("hotchpotch/jaqket_v1_qa_wikija_context")  # type: ignore
valid_ds = ds["validation"]  # type: ignore
valid_df = valid_ds.data.to_pandas()  # type: ignore
# context は list なので、 "\n" で結合する
valid_df["context"] = valid_df["context"].apply(lambda x: "\n".join(x) + "\n")
valid_df.head(1)

,qid,question,answer,context,answers,competition,timestamp,section,number,original_question,original_answer,original_additional_info
0,QA20QBIK-0912,手術によって天才となったチャーリイ・ゴードンの日記という形式を通して語られる、ダニエル・キイ...,アルジャーノンに花束を,ダニエル・キイス 「アルジャーノンに花束を」は中編もそれを発展させた長編も、知的障害者のチャ...,[アルジャーノンに花束を],第1回AI王,2020/01/27,開発データ問題 (dev1),912,手術によって天才となったチャーリイ・ゴードンの日記という形式を通して語られる、ダニエル・キイ...,アルジャーノンに花束を,


In [6]:
# valid_df での正解率を測る
from tqdm import tqdm

# valid_df = valid_df.head(10)

valid_df = valid_df.reset_index(drop=True)
for i in tqdm(range(len(valid_df))):
    pred_text = qa(
        model, tokenizer, valid_df.loc[i, "question"], valid_df.loc[i, "context"]
    )
    valid_df.loc[i, "pred"] = pred_text

100%|██████████| 980/980 [05:01<00:00,  3.25it/s]


In [7]:
# 完全一致の正解率を表示
valid_df["is_correct"] = valid_df["answer"] == valid_df["pred"]
valid_df["is_correct"].mean()

0.7316326530612245

In [8]:
# 間違ったものだけを表示
incorrect_df = valid_df[["question", "answer", "pred", "is_correct"]][
    valid_df["is_correct"] == False
]
incorrect_df

,question,answer,pred,is_correct
0,手術によって天才となったチャーリイ・ゴードンの日記という形式を通して語られる、ダニエル・キイ...,アルジャーノンに花束を,アルジャーノンに花束を』,False
3,現代の「世界三大ギタリスト」と称されるのはデレク・トラックス、ジョン・フルシアンテと誰?,ジョン・メイヤー,ジョン・フルシアンテ,False
4,テレビ番組の検証映像でもよく使用される、映した物体の温度を色分けして表示する装置を何というで...,サーモグラフィー,サーモグラフィーカメラ,False
5,不景気にもかかわらず、原材料の高騰などで物価水準が上昇し続ける状態のことを経済用語で何という?,スタグフレーション,デフレーション,False
7,古代ギリシャの彫刻が起源とされ、飛鳥時代の仏像にも見られる、口元に微笑みを浮かべたように見え...,アルカイク・スマイル,アルカイック・スマイル,False
...,...,...,...,...
966,2020年シーズン、読売ジャイアンツの丸佳浩が試合での登場時に主に流していたのは、何という歌...,LiSA,Rising Hope』,False
967,2人で行う花札の遊び方で、「五光」や「猪鹿蝶」といった役があるのは何?,こいこい,花札,False
968,かつて電気が無い時代に、最後の出演者がろうそくの芯を叩いて火を消していたことに由来する、落語...,真打,真打ち,False
971,アンデルセンの『人魚姫』を題材にした、1989年公開のディズニー映画は何でしょう?,リトル・マーメイド,リトル・マーメイド』,False


In [9]:
# 部分一致の正解率を表示
valid_df["is_correct"] = valid_df.apply(lambda x: x["answer"] in x["pred"], axis=1)
valid_df["is_correct"].mean()

0.8846938775510204

In [10]:
# 間違ったものだけを表示
valid_df[["question", "answer", "pred", "is_correct"]][valid_df["is_correct"] == False]

,question,answer,pred,is_correct
3,現代の「世界三大ギタリスト」と称されるのはデレク・トラックス、ジョン・フルシアンテと誰?,ジョン・メイヤー,ジョン・フルシアンテ,False
5,不景気にもかかわらず、原材料の高騰などで物価水準が上昇し続ける状態のことを経済用語で何という?,スタグフレーション,デフレーション,False
7,古代ギリシャの彫刻が起源とされ、飛鳥時代の仏像にも見られる、口元に微笑みを浮かべたように見え...,アルカイク・スマイル,アルカイック・スマイル,False
10,本場鹿児島では「つけあげ」と呼ばれる揚げ物は何でしょう?,薩摩揚げ,つけあげ,False
14,飛行機の操縦士としての体験を元に小説『夜間飛行』や童話『星の王子様』などの作品を残したフラン...,アントワーヌ・ド・サン=テグジュペリ,アントワーヌ・ド・サン＝テグジュペリ,False
...,...,...,...,...
944,「ちょっと何言ってるか分からない」というセリフで知られる、お笑いコンビ・サンドウィッチマンの...,富澤たけし,伊達みきお,False
964,旅客機の座席で、ファーストクラスとエコノミークラスの中間にあたるものを一般に何クラスというで...,ビジネスクラス,エコノミークラス,False
966,2020年シーズン、読売ジャイアンツの丸佳浩が試合での登場時に主に流していたのは、何という歌...,LiSA,Rising Hope』,False
967,2人で行う花札の遊び方で、「五光」や「猪鹿蝶」といった役があるのは何?,こいこい,花札,False


In [11]:
print(
    valid_df[["question", "answer", "pred", "is_correct"]][
        valid_df["is_correct"] == False
    ].to_csv()
)

,question,answer,pred,is_correct
3,現代の「世界三大ギタリスト」と称されるのはデレク・トラックス、ジョン・フルシアンテと誰?,ジョン・メイヤー,ジョン・フルシアンテ,False
5,不景気にもかかわらず、原材料の高騰などで物価水準が上昇し続ける状態のことを経済用語で何という?,スタグフレーション,デフレーション,False
7,古代ギリシャの彫刻が起源とされ、飛鳥時代の仏像にも見られる、口元に微笑みを浮かべたように見える表情を何というでしょう?,アルカイク・スマイル,アルカイック・スマイル,False
10,本場鹿児島では「つけあげ」と呼ばれる揚げ物は何でしょう?,薩摩揚げ,つけあげ,False
14,飛行機の操縦士としての体験を元に小説『夜間飛行』や童話『星の王子様』などの作品を残したフランスの作家は誰でしょう?,アントワーヌ・ド・サン=テグジュペリ,アントワーヌ・ド・サン＝テグジュペリ,False
22,日本銀行法第8条で、「一億円とする」と定められているのは、日本銀行に関するどんなものについてでしょう?,資本金,日本銀行法,False
28,鎌倉幕府や室町幕府が荘園を管理支配するために設置した役職で、道理の通じない相手の例えとして「泣く子とコレには勝てぬ」ともいわれるのは何でしょう?,地頭,荘官,False
50,クラシック音楽の種類で、「シンフォニー」は日本語で交響曲ですが、「コンチェルト」は日本語で何というでしょう?,協奏曲,コンチェルト」,False
58,山号を「瑞龍山」という、足利義満の時代に、京都五山・鎌倉五山の別格上位に列せられた京都市左京区の寺は何?,南禅寺,天龍寺,False
63,パーツ、グラビア、読者、ファッションの後ろに共通してつく言葉は何?,モデル,ファッション,False
72,兵庫県の「岩津」、埼玉県の「深谷」、群馬県の「下仁田」といえばどんな野菜のブランドでしょう?,ネギ,深谷ねぎ,False
83,ティーンエイジャー向けに書かれた小説のことで、英語で「少年向け」という意味があるジャンルを何小説という?,ジュブナイル,ライトノベル,False
99,アフリカの国・シエラレオネの国旗と同じ色の並びの看板を使っている、日本のコンビニチェーンは何?,ファミリーマート,セブン-イレブン,False
10